In [ ]:
!pip install xgboost
!pip install category_encoders
!pip install catboost

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 72 kB 893 kB/s 


In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import cross_val_score, train_test_split
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression
from sklearn.metrics import SCORERS, mean_absolute_error
from xgboost import XGBRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler 
from sklearn.preprocessing import LabelEncoder
import sys
from category_encoders import *
from pandas_profiling import ProfileReport

In [ ]:
train = pd.read_csv("/content/drive/MyDrive/dacon/train.csv")

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/dacon/test.csv")

In [ ]:
train.isnull().sum()

id                       0
base_date                0
day_of_week              0
base_hour                0
road_in_use              0
lane_count               0
road_rating              0
road_name                0
multi_linked             0
connect_code             0
maximum_speed_limit      0
vehicle_restricted       0
weight_restricted        0
height_restricted        0
road_type                0
start_node_name          0
start_latitude           0
start_longitude          0
start_turn_restricted    0
end_node_name            0
end_latitude             0
end_longitude            0
end_turn_restricted      0
target                   0
dtype: int64

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4701217 entries, 0 to 4701216
Data columns (total 24 columns):
 #   Column                 Dtype  
---  ------                 -----  
 0   id                     object 
 1   base_date              int64  
 2   day_of_week            object 
 3   base_hour              int64  
 4   road_in_use            int64  
 5   lane_count             int64  
 6   road_rating            int64  
 7   road_name              object 
 8   multi_linked           int64  
 9   connect_code           int64  
 10  maximum_speed_limit    float64
 11  vehicle_restricted     float64
 12  weight_restricted      float64
 13  height_restricted      float64
 14  road_type              int64  
 15  start_node_name        object 
 16  start_latitude         float64
 17  start_longitude        float64
 18  start_turn_restricted  object 
 19  end_node_name          object 
 20  end_latitude           float64
 21  end_longitude          float64
 22  end_turn_restricte

In [ ]:
train.head()

,id,base_date,day_of_week,base_hour,road_in_use,lane_count,road_rating,road_name,multi_linked,connect_code,...,road_type,start_node_name,start_latitude,start_longitude,start_turn_restricted,end_node_name,end_latitude,end_longitude,end_turn_restricted,target
0,TRAIN_0000000,20220623,목,17,0,1,106,지방도1112호선,0,0,...,3,제3교래교,33.427747,126.662612,없음,제3교래교,33.427749,126.662335,없음,52.0
1,TRAIN_0000001,20220728,목,21,0,2,103,일반국도11호선,0,0,...,0,광양사거리,33.500730,126.529107,있음,KAL사거리,33.504811,126.526240,없음,30.0
2,TRAIN_0000002,20211010,일,7,0,2,103,일반국도16호선,0,0,...,0,창고천교,33.279145,126.368598,없음,상창육교,33.280072,126.362147,없음,61.0
3,TRAIN_0000003,20220311,금,13,0,2,107,태평로,0,0,...,0,남양리조트,33.246081,126.567204,없음,서현주택,33.245565,126.566228,없음,20.0
4,TRAIN_0000004,20211005,화,8,0,2,103,일반국도12호선,0,0,...,0,애월샷시,33.462214,126.326551,없음,애월입구,33.462677,126.330152,없음,38.0


In [ ]:
## DF
col1 target   
 A.    10
 B.    15
 B.    35
 C.    40

col1 target
 10. 10
 25. 15
 25. 35
 40. 40 

A B C -> 1 2 3

In [ ]:
from category_encoders import target_encoder

In [ ]:
info = pd.read_csv("/content/drive/MyDrive/dacon/data_info.csv")
info

,변수명,변수 설명
0,id,아이디
1,base_date,날짜
2,day_of_week,요일
3,base_hour,시간대
4,road_in_use,도로사용여부
5,lane_count,차로수
6,road_rating,도로등급
7,multi_linked,중용구간 여부
8,connect_code,연결로 코드
9,maximum_speed_limit,최고속도제한


In [ ]:
train.describe()

,base_date,base_hour,road_in_use,lane_count,road_rating,multi_linked,connect_code,maximum_speed_limit,vehicle_restricted,weight_restricted,height_restricted,road_type,start_latitude,start_longitude,end_latitude,end_longitude,target
count,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4701217.0,4.701217e+06,4701217.0,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06,4.701217e+06
mean,2.021672e+07,1.192820e+01,1.362413e-03,1.836651e+00,1.049585e+02,4.762597e-04,2.660218e-01,6.125329e+01,0.0,5.618742e+03,0.0,6.152237e-01,3.338432e+01,1.265217e+02,3.338432e+01,1.265217e+02,4.278844e+01
std,4.555709e+03,6.722092e+00,3.688573e-02,6.877513e-01,1.840107e+00,2.181818e-02,5.227760e+00,1.213354e+01,0.0,1.395343e+04,0.0,1.211268e+00,1.012015e-01,1.563657e-01,1.011948e-01,1.563519e-01,1.595443e+01
min,2.021090e+07,0.000000e+00,0.000000e+00,1.000000e+00,1.030000e+02,0.000000e+00,0.000000e+00,3.000000e+01,0.0,0.000000e+00,0.0,0.000000e+00,3.324343e+01,1.261826e+02,3.324343e+01,1.261826e+02,1.000000e+00
25%,2.021110e+07,6.000000e+00,0.000000e+00,1.000000e+00,1.030000e+02,0.000000e+00,0.000000e+00,5.000000e+01,0.0,0.000000e+00,0.0,0.000000e+00,3.326422e+01,1.264232e+02,3.326422e+01,1.264232e+02,3.000000e+01
50%,2.022013e+07,1.200000e+01,0.000000e+00,2.000000e+00,1.060000e+02,0.000000e+00,0.000000e+00,6.000000e+01,0.0,0.000000e+00,0.0,0.000000e+00,3.341257e+01,1.265112e+02,3.341257e+01,1.265112e+02,4.300000e+01
75%,2.022050e+07,1.800000e+01,0.000000e+00,2.000000e+00,1.070000e+02,0.000000e+00,0.000000e+00,7.000000e+01,0.0,0.000000e+00,0.0,0.000000e+00,3.347804e+01,1.265840e+02,3.347804e+01,1.265840e+02,5.400000e+01
max,2.022073e+07,2.300000e+01,1.000000e+00,3.000000e+00,1.070000e+02,1.000000e+00,1.030000e+02,8.000000e+01,0.0,5.000000e+04,0.0,3.000000e+00,3.355608e+01,1.269309e+02,3.355608e+01,1.269309e+02,1.130000e+02


In [ ]:
train[['base_hour','target']].groupby('base_hour').mean()

,target
base_hour,
0,48.742200
1,49.763300
2,50.278658
3,50.543254
4,49.747525
5,47.201140
6,45.549029
7,43.682268
8,41.656292


In [ ]:
train[['lane_count','target']].groupby('lane_count').mean()

,target
lane_count,
1,43.570562
2,44.915713
3,34.917783


In [ ]:
train[['multi_linked','target']].groupby('multi_linked').mean()

,target
multi_linked,
0,42.791370
1,36.642698


In [ ]:
train[['connect_code','target']].groupby('connect_code').mean()

,target
connect_code,
0,42.749190
103,57.947043


In [ ]:
train[['maximum_speed_limit','target']].groupby('maximum_speed_limit').mean()

,target
maximum_speed_limit,
30.0,32.220825
40.0,62.774080
50.0,35.492231
60.0,42.316287
70.0,41.366208
80.0,60.703696


In [ ]:
train[['weight_restricted','target']].groupby('weight_restricted').mean()

,target
weight_restricted,
0.0,40.955696
32400.0,47.453350
43200.0,57.883888
50000.0,47.088665


In [ ]:
train[['road_type','target']].groupby('road_type').mean()

,target
road_type,
0,41.160930
3,49.097125


In [ ]:
def group_hour(time):
  res = -1
  if 0 < time < 5:
    res = 1
  elif time < 9:
    res = 5
  elif time < 13:
    res = 9
  elif time < 17:
    res = 13
  elif time < 21:
    res = 17
  else:
    res = 21
  return res

In [ ]:
target_enc = TargetEncoder(return_df=True)
minmax_scaler = MinMaxScaler()

/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:124: FutureWarning: Default parameter min_samples_leaf will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)
/usr/local/lib/python3.7/dist-packages/category_encoders/target_encoder.py:129: FutureWarning: Default parameter smoothing will change in version 2.6.See https://github.com/scikit-learn-contrib/category_encoders/issues/327
  category=FutureWarning)


In [ ]:
def preprocess(input:pd.DataFrame, test_set=False) -> pd.DataFrame:
  
  data = input.copy()
  # 하루를 4시간 단위로 그룹화
  #data['base_hour'] = data['base_hour'].apply(lambda x: group_hour(x))
  # 주중/주말 나누기
  #dic = {"월":0, "화":0, "수":0, "목":0, "금":0, "토":1, "일":1}
  #data['day_of_week'] = data['day_of_week'].apply(lambda x: dic[x])
  # 필요없는 열 정리하기
  data.drop(['road_in_use','id', 'base_date','vehicle_restricted','height_restricted'], axis=1, inplace=True)
  # True/False 인코딩
  data['start_turn_restricted'] = data['start_turn_restricted'].apply(lambda x: 1 if x=="있음" else 0)
  data['end_turn_restricted'] = data['end_turn_restricted'].apply(lambda x: 1 if x=="있음" else 0)

  data['start_node_name'] = data['start_node_name'].astype('category')
  #data['end_node_name'] = encoder.fit_transform(data['end_node_name'])
  data['end_node_name'] = data['end_node_name'].astype('category')
  data['day_of_week'] = data['day_of_week'].astype('category')
  data['base_hour'] = data['base_hour'].astype('str')
  data['end_latitude'] = data['end_latitude'].astype('str')
  data['end_longitude'] = data['end_longitude'].astype('str')
  data['start_latitude'] = data['start_latitude'].astype('str')
  data['end_latitude'] = data['end_latitude'].astype('str')
  data['road_rating'] = data['road_rating'].astype('category')
  data['road_type'] = data['road_type'].astype('category')

  if test_set:
    return minmax_scaler.transform(target_enc.transform(data))

  else:
    X = data.drop(['target'], axis=1)
    y = data['target']
    X = minmax_scaler.fit_transform(target_enc.fit_transform(X, y))
    return X,y

In [ ]:
X, y = preprocess(train)

In [ ]:
test_x = preprocess(test, test_set=True)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.05, random_state=10)

In [ ]:
print("train length:", len(X_train))
print("test length:",len(X_test))

train length: 4466156
test length: 235061


In [ ]:
sys.getsizeof(X_train)/8000000

80.390823

In [ ]:
model = XGBRegressor(n_estimators=10000, learning_rate=0.02,colsample_bytree=1, gamma=3, max_depth=6,reg_alpha=20, reg_lambda=2,
                     tree_method='gpu_hist', gpu_id=0)
model.fit(X_train, y_train,eval_metric='mae',early_stopping_rounds=10, eval_set=[(X_test, y_test)])

스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[5000]	validation_0-mae:3.61431
[5001]	validation_0-mae:3.61429
[5002]	validation_0-mae:3.61428
[5003]	validation_0-mae:3.61427
[5004]	validation_0-mae:3.61425
[5005]	validation_0-mae:3.61422
[5006]	validation_0-mae:3.61422
[5007]	validation_0-mae:3.61422
[5008]	validation_0-mae:3.61419
[5009]	validation_0-mae:3.61418
[5010]	validation_0-mae:3.61415
[5011]	validation_0-mae:3.61415
[5012]	validation_0-mae:3.61413
[5013]	validation_0-mae:3.61411
[5014]	validation_0-mae:3.6141
[5015]	validation_0-mae:3.61408
[5016]	validation_0-mae:3.61406
[5017]	validation_0-mae:3.61405
[5018]	validation_0-mae:3.61403
[5019]	validation_0-mae:3.61402
[5020]	validation_0-mae:3.614
[5021]	validation_0-mae:3.61398
[5022]	validation_0-mae:3.61395
[5023]	validation_0-mae:3.61392
[5024]	validation_0-mae:3.61389
[5025]	validation_0-mae:3.61386
[5026]	validation_0-mae:3.61384
[5027]	validation_0-mae:3.61383
[5028]	validation_0-mae:3.6138
[5029]	validation_0-mae:3.61377
[5030]	v

XGBRegressor(gamma=3, gpu_id=0, learning_rate=0.02, max_depth=6,
             n_estimators=10000, reg_alpha=20, reg_lambda=2,
             tree_method='gpu_hist')

In [ ]:
import sys

In [ ]:
sys.getsizeof(6)

28

In [ ]:
# 4.17579 (요일/시간대 조정했을 때)
# 4.14238 (row 데이터 그대로) -> model 1 (10000 epoch 3.9)
# 3.87855 위도 경도 다 포함시켰을때 -> model 2
# model2 (10000 epoch )
# model2 + bayesian optimization 

In [ ]:
import joblib
filename = 'xgboost_model_1.pkl' 
joblib.dump(model, filename)

['xgboost_model_1.pkl']

In [ ]:
#모델 불러오기
xgb = joblib.load(filename) 

[08:35:05] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [ ]:
from catboost import CatBoostRegressor

In [ ]:
cat_model = CatBoostRegressor(learning_rate=0.1, iterations=3000, loss_function="MAE", random_state = 10,task_type="GPU")

In [ ]:
cat_model.fit(X_train, y_train, cat_features=np.where(X_train.dtypes != np.float64)[0], eval_set=(X_test, y_test))

Default metric period is 5 because MAE is/are not implemented for GPU


0:	learn: 13.1879916	test: 13.1765531	best: 13.1765531 (0)	total: 591ms	remaining: 29m 31s
1:	total: 1.15s	remaining: 28m 43s
2:	total: 1.66s	remaining: 27m 36s
3:	total: 2.17s	remaining: 27m 2s
4:	total: 2.73s	remaining: 27m 13s
5:	learn: 13.0166559	test: 13.0048583	best: 13.0048583 (5)	total: 3.32s	remaining: 27m 36s
6:	total: 3.9s	remaining: 27m 46s
7:	total: 4.45s	remaining: 27m 42s
8:	total: 5.12s	remaining: 28m 21s
9:	total: 5.74s	remaining: 28m 35s
10:	learn: 12.8451886	test: 12.8330231	best: 12.8330231 (10)	total: 6.25s	remaining: 28m 17s
11:	total: 6.87s	remaining: 28m 29s
12:	total: 7.48s	remaining: 28m 39s
13:	total: 8.06s	remaining: 28m 39s
14:	total: 8.65s	remaining: 28m 42s
15:	learn: 12.6735922	test: 12.6610784	best: 12.6610784 (15)	total: 9.2s	remaining: 28m 36s
16:	total: 9.92s	remaining: 29m
17:	total: 10.5s	remaining: 29m 5s
18:	total: 11.2s	remaining: 29m 12s
19:	total: 11.7s	remaining: 29m 6s
20:	learn: 12.5017845	test: 12.4887689	best: 12.4887689 (20)	total: 12.3s

## Random Forest

In [ ]:
model_forest = RandomForestRegressor(n_estimators=100,
                                     max_depth=5,
                                     criterion="absolute_error",
                                     n_jobs=-1)
model_forest.fit(X_train, y_train)
model_forest.predict(X_test)
socre = mean_absolute_error(y_test, pred)
print("forest score : ", score)

KeyboardInterrupt: ignored

## Bayesian Optimization

In [ ]:
from hyperopt import STATUS_OK, Trials, fmin, hp, tpe

In [ ]:
space={'max_depth': hp.quniform("max_depth", 3, 18, 1),
        'gamma': hp.uniform ('gamma', 1,9),
        'reg_alpha' : hp.quniform('reg_alpha', 40,180,1),
        'reg_lambda' : hp.uniform('reg_lambda', 0,1),
        'colsample_bytree' : hp.uniform('colsample_bytree', 0.5,1),
        'min_child_weight' : hp.quniform('min_child_weight', 0, 10, 1),
        'n_estimators': 200,
        'seed': 0
    }

In [ ]:
def objective(space):
    clf=XGBRegressor(
                    n_estimators =int(space['n_estimators']), max_depth = int(space['max_depth']), gamma = space['gamma'],
                    reg_alpha = int(space['reg_alpha']),reg_lambda = space['reg_lambda'],min_child_weight=int(space['min_child_weight']),
                    colsample_bytree=(space['colsample_bytree']),tree_method='gpu_hist', gpu_id=0)
    
    evaluation = [( X_train, y_train), ( X_test, y_test)]
    
    clf.fit(X_train, y_train,
            eval_set=evaluation, eval_metric="mae",
            early_stopping_rounds=10,verbose=False)
    

    pred = clf.predict(X_test)
    mae = mean_absolute_error(y_test, pred)
    if mae < 6.5:
      print("SCORE:", mae)
    return {'loss': mae, 'status': STATUS_OK }

In [ ]:
trials = Trials()

best_hyperparams = fmin(fn = objective,
                        space = space,
                        algo = tpe.suggest,
                        max_evals = 100,
                        trials = trials)

[08:45:27] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
3.5187657252407654
[08:46:42] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
3.762656396168691
[08:46:59] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
3.61225304180884
[08:47:22] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
3.8192513469581457
[08:47:40] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
3.771121163350597
[08:47:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
SCORE:
4.122738062236189
[08:48:13] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now d

KeyboardInterrupt: ignored

In [ ]:
print(best_hyperparams)

NameError: ignored

In [ ]:
def make_submission(model, test_data) -> None:
  pred = model.predict(test_data)
  temp = pd.DataFrame(columns=['id', 'target'], index=test.index)
  temp['id'] = test['id']
  temp['target'] = pred
  
  temp.to_csv("res.csv", index=False)
  print("success!!")

In [ ]:
make_submission(model, test_x)

success!!
